# Pf7 Data Access 

This page provides information about how to access data from [Plasmodium falciparum version 7 (Pf7)](https://www.malariagen.net/resource/34)  project via Google Cloud. This includes sample metadata and single nucleotide polymorphism (SNP) calls. This release spans multiple [MalariaGEN](https://www.malariagen.net/) projects including [Pf community project](https://www.malariagen.net/parasite/p-falciparum-community-project), [GenRe-Mekong](https://www.malariagen.net/parasite/genre-mekong) and [SpotMalaria](https://www.malariagen.net/parasite/spotmalaria), and a collaboration between xx studies spread around the globe. 

This notebook illustrates how to read data directly from the cloud, **without having to first download any data locally**. This notebook can be run from any computer, but will work best when run from a compute node within Google Cloud, because it will be physically closer to the data and so data transfer is faster. For example, this notebook can be run via MyBinder or Google Colab which are free interactive computing service running in the cloud.

To launch this notebook in the cloud and run it for yourself, click the launch icon () at the top of the page and select one of the cloud computing services available.

For a quick overview on the spatial and geographical distribution of the samples available, please visit our [Pf7 web-app](https://www.malariagen.net/apps/pf7/). 

## Setup

Running this notebook requires some Python packages to be installed. These packages can be installed via pip or conda. E.g.:

In [ ]:
# The pf7 module is not yet included in the malariagen package. When it is the below should work.
# !pip install -q malariagen_data

To make accessing these data more convenient, we’ve created the malariagen_data Python package, which is available from PyPI. This is experimental so please let us know if you find any bugs or have any suggestions.

Now import the packages we’ll need to use here.

In [1]:
from malariagen_data.pf7 import * 
# import malariagen_data (above line will eventually be substituted for this)

import numpy as np
import dask
import dask.array as da
from dask.diagnostics.progress import ProgressBar
# silence some warnings
dask.config.set(**{'array.slicing.split_large_chunks': False})
import allel

To access the pf7 data stored on google cloud use the following code:

In [2]:
pf7 = Pf7("gs://pf7_staging/")

## Metadata

Data on the samples that were sequenced as part of this resource are available. It includes the time and place of collection, quality metrics, and accesion numbers.

To see all the information available, load sample metadata into a pandas dataframe:

In [3]:
pf7_metadata = pf7.sample_metadata()

pf7_metadata.head()

,Sample,Study,Country,Admin level 1,Country latitude,Country longitude,Admin level 1 latitude,Admin level 1 longitude,Year,ENA,All samples same case,Population,% callable,QC pass,Exclusion reason,Sample type,Sample was in Pf6
0,FP0008-C,1147-PF-MR-CONWAY,Mauritania,Hodh el Gharbi,20.265149,-10.337093,16.565426,-9.832345,2014.0,ERR1081237,FP0008-C,AF-W,82.16,True,Analysis_set,gDNA,True
1,FP0009-C,1147-PF-MR-CONWAY,Mauritania,Hodh el Gharbi,20.265149,-10.337093,16.565426,-9.832345,2014.0,ERR1081238,FP0009-C,AF-W,88.85,True,Analysis_set,gDNA,True
2,FP0010-CW,1147-PF-MR-CONWAY,Mauritania,Hodh el Gharbi,20.265149,-10.337093,16.565426,-9.832345,2014.0,ERR2889621,FP0010-CW,AF-W,86.46,True,Analysis_set,sWGA,False
3,FP0011-CW,1147-PF-MR-CONWAY,Mauritania,Hodh el Gharbi,20.265149,-10.337093,16.565426,-9.832345,2014.0,ERR2889624,FP0011-CW,AF-W,86.35,True,Analysis_set,sWGA,False
4,FP0012-CW,1147-PF-MR-CONWAY,Mauritania,Hodh el Gharbi,20.265149,-10.337093,16.565426,-9.832345,2014.0,ERR2889627,FP0012-CW,AF-W,89.74,True,Analysis_set,sWGA,False


In [4]:
print("The data set has {} samples and {} fields".format(pf7_metadata.shape[0],pf7_metadata.shape[1]))

The data set has 20864 samples and 17 fields


We can explore each of the fields: 

- The <font color='purple'>Sample</font> column gives the unique sample identifier used throughout all Pf7 analyses.


- The <font color='purple'>Study</font> refers to the partner study which collected the sample.


- The <font color='purple'>Country</font> & <font color='purple'>Admin level 1</font> describe the location where the sample was collected from. 


- The <font color='purple'>Country latitude</font>, <font color='purple'>Country longitude</font>, <font color='purple'>Admin level 1 latitude</font> and <font color='purple'>Admin 1 longitude</font> contain the GADM coordinates for each country & administrative level 1.


- The <font color='purple'>Year</font> column gives the time of sample collection.


- The <font color='purple'>ENA</font> column gives the run accession(s) for the sequencing read data for each sample.


- The <font color='purple'>All samples same case</font> column identifies samples set collected from the same individual. 
    
    
- The <font color='purple'>Population</font> column gives the population to which the sample has been assigned to, based on the degree of genetic similarity to other samples. The possible values are: Africa-Central (AF-C), Africa - East (AF-E), Africa - Northeast (AF-NE), Africa - West (AF-W), Asia - South Asia - East (AS-SA-E), Asia - South Asia - West (AS-SA-W), Asia - Southeast Asia - East (AS-SEA-E), Asia - Southeast Asia - West (AS-SEA-W), Oceania - New Guinea (OC-NG), South America (SA). 


- The <font color='purple'>% callable</font> column refers to the %  of the genome with coverage of at least 5 reads and less than 10% of reads with mapping quality 0. 
 
 
- The <font color='purple'>QC pass</font> column defines whether the sample passed (True) or failed (False) QC. 
    
    
- The <font color='purple'>Exclusion reason</font> describes the reason why the particular sample was excluded from the main analysis.
    
    
- The <font color='purple'>Sample type</font> column gives details on the sequencing method used on the particular sample  (gDNA, sWGA, MDA).
    
    
- The <font color='purple'>Sample was in Pf6</font> column defines whether the sample was included in the previous version of the data release (Pf6) or if it is new to Pf7.

The python package [Pandas](https://pandas.pydata.org/) can be used to explore and query the sample metadata in different ways. For example, here is a summary of the numbers of samples grouped by the country they were collected in:

In [5]:
pf7_metadata.groupby("Country").size()

Country
Bangladesh                          1661
Benin                                334
Burkina Faso                          58
Cambodia                            1726
Cameroon                             295
Colombia                             164
Côte d'Ivoire                         71
Democratic Republic of the Congo     573
Ethiopia                              34
Gabon                                 59
Gambia                              1247
Ghana                               4150
Guinea                               199
India                                318
Indonesia                            135
Kenya                                732
Laos                                1052
Madagascar                            25
Malawi                               371
Mali                                1804
Mauritania                           104
Mozambique                            92
Myanmar                             1261
Nigeria                              146
Papua Ne

## Variant Calls 

These files contain details of 10,145,661 discovered variant genome positions. These variants were discovered amongst all samples from the release.

4,397,801 of these variant positions are SNPs, with the remainder being either short insertion/deletions (indels), or a combination of SNPs and indels. 

Data on variant calls, including the genomic positions, alleles, and genotypes, can be accessed as an xarray Dataset:

In [6]:
variant_dataset = pf7.variant_calls()
variant_dataset

<xarray.Dataset>
Dimensions:              (alleles: 7, ploidy: 2, samples: 20864, variants: 10145661)
Coordinates:
    variant_position     (variants) int32 dask.array<chunksize=(8388608,), meta=np.ndarray>
    variant_chrom        (variants) object dask.array<chunksize=(4194304,), meta=np.ndarray>
    sample_id            (samples) object dask.array<chunksize=(20864,), meta=np.ndarray>
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    variant_allele       (variants, alleles) object dask.array<chunksize=(699051, 1), meta=np.ndarray>
    variant_filter_pass  (variants) bool dask.array<chunksize=(10145661,), meta=np.ndarray>
    variant_is_snp       (variants) bool dask.array<chunksize=(10145661,), meta=np.ndarray>
    variant_numalt       (variants) int32 dask.array<chunksize=(8388608,), meta=np.ndarray>
    variant_CDS          (variants) bool dask.array<chunksize=(10145661,), meta=np.ndarray>
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(167773, 100, 2), meta=np.ndarray>
    call_AD              (variants, samples, alleles) int16 dask.array<chunksize=(23968, 100, 7), meta=np.ndarray>

The default returns a basic set of data most commonly used for data analysis. However, for more complex analysis the full range of variables available in the zarr can be accessed by setting the extended flag to True, as shown below: 

In [7]:
extended_variant_dataset = pf7.variant_calls(extended=True)
extended_variant_dataset

<xarray.Dataset>
Dimensions:                                        (alleles: 7, alt_alleles: 6, genotypes: 3, ploidy: 2, samples: 20864, sb_statistics: 4, variants: 10145661)
Coordinates:
    variant_position                               (variants) int32 dask.array<chunksize=(8388608,), meta=np.ndarray>
    variant_chrom                                  (variants) object dask.array<chunksize=(4194304,), meta=np.ndarray>
    sample_id                                      (samples) object dask.array<chunksize=(20864,), meta=np.ndarray>
Dimensions without coordinates: alleles, alt_alleles, genotypes, ploidy, samples, sb_statistics, variants
Data variables: (12/92)
    variant_allele                                 (variants, alleles) object dask.array<chunksize=(699051, 1), meta=np.ndarray>
    variant_filter_pass                            (variants) bool dask.array<chunksize=(10145661,), meta=np.ndarray>
    variant_is_snp                                 (variants) bool dask.array<chunksize=(10145661,), meta=np.ndarray>
    variant_numalt                                 (variants) int32 dask.array<chunksize=(8388608,), meta=np.ndarray>
    variant_CDS                                    (variants) bool dask.array<chunksize=(10145661,), meta=np.ndarray>
    call_genotype                                  (variants, samples, ploidy) int8 dask.array<chunksize=(167773, 100, 2), meta=np.ndarray>
    ...                                             ...
    variant_RegionType                             (variants) object dask.array<chunksize=(4194304,), meta=np.ndarray>
    variant_SOR                                    (variants) float32 dask.array<chunksize=(8388608,), meta=np.ndarray>
    variant_VQSLOD                                 (variants) float32 dask.array<chunksize=(8388608,), meta=np.ndarray>
    variant_altlen                                 (variants, alt_alleles) int32 dask.array<chunksize=(1398102, 6), meta=np.ndarray>
    variant_culprit                                (variants) object dask.array<chunksize=(4194304,), meta=np.ndarray>
    variant_set                                    (variants) object dask.array<chunksize=(4194304,), meta=np.ndarray>

Each of the elements in this xarray is a dask array. The individual dask arrays can be accessed as follows, replacing the string with the variable you are looking for:

In [8]:
pos = variant_dataset["variant_position"].data
pos

dask.array<array, shape=(10145661,), dtype=int32, chunksize=(8388608,), chunktype=numpy.ndarray>

# Genotypes

Genotypes for individual samples are available. 

Genotypes are stored as a three-dimensional array, where:
- the first dimension corresponds to genomic positions, 
- the second dimension is samples,
- the third dimension is ploidy (2).

Values coded as integers, where -1 represents a missing value, 0 represents the reference allele, and 1, 2, and 3 represent alternate alleles.

Variant genotypes can be accessed as dask arrays as shown below.

In [9]:
gt = variant_dataset["call_genotype"].data
gt

,Array,Chunk
Bytes,423.36 GB,33.55 MB
Shape,"(10145661, 20864, 2)","(167773, 100, 2)"
Count,12749 Tasks,12749 Chunks
Type,int8,numpy.ndarray


Note that the columns of this array (second dimension) match the rows in the sample metadata. You can use this correspondance to apply further subsetting operations to the genotypes by querying the sample metadata. E.g.:

In [10]:
loc_colombia = pf7_metadata.eval("Country == 'Colombia'").values
print(f"found {np.count_nonzero(loc_colombia)} samples from Colombia")
variant_dataset_colombia = variant_dataset.isel(samples=loc_colombia)
variant_dataset_colombia

found 164 samples from Colombia


<xarray.Dataset>
Dimensions:              (alleles: 7, ploidy: 2, samples: 164, variants: 10145661)
Coordinates:
    variant_position     (variants) int32 dask.array<chunksize=(8388608,), meta=np.ndarray>
    variant_chrom        (variants) object dask.array<chunksize=(4194304,), meta=np.ndarray>
    sample_id            (samples) object dask.array<chunksize=(164,), meta=np.ndarray>
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    variant_allele       (variants, alleles) object dask.array<chunksize=(699051, 1), meta=np.ndarray>
    variant_filter_pass  (variants) bool dask.array<chunksize=(10145661,), meta=np.ndarray>
    variant_is_snp       (variants) bool dask.array<chunksize=(10145661,), meta=np.ndarray>
    variant_numalt       (variants) int32 dask.array<chunksize=(8388608,), meta=np.ndarray>
    variant_CDS          (variants) bool dask.array<chunksize=(10145661,), meta=np.ndarray>
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(167773, 32, 2), meta=np.ndarray>
    call_AD              (variants, samples, alleles) int16 dask.array<chunksize=(23968, 32, 7), meta=np.ndarray>

The data on genomic variants can be loaded into memory as [numpy](https://numpy.org/doc/stable/reference/generated/numpy.array.html) arrays as shown in the following examples.

In [11]:
g = gt[:5, :3, :].compute()
g

array([[[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]],

       [[-1, -1],
        [ 0,  0],
        [-1, -1]]], dtype=int8)

If you want to work with the genotype calls, you may find it convenient to use scikit-allel. E.g., the code below sets up a genotype array.

In [12]:
# use the scikit-allel wrapper class for genotype calls
gt = allel.GenotypeDaskArray(variant_dataset_colombia["call_genotype"].data)
gt

<GenotypeDaskArray shape=(10145661, 164, 2) dtype=int8>